In [89]:
import numpy as np # type: ignore
import sys
import os

from scipy.linalg import qr # type: ignore
from scipy.linalg import svd # type: ignore
import numpy as np # type: ignore
import transforms # type: ignore

import time
import random

print(sys.executable)

/opt/homebrew/Caskroom/miniconda/base/envs/rand_dec/bin/python


In [90]:
def my_matmul(A, B):
  n, m = A.shape
  _, k = B.shape
  C = np.zeros((n, k))
  for i in range(0, n):
    for j in range(0, k):
      for l in range(0, m):
        C[i][j] += A[i][l] * B[l][j]
  return C

In [91]:
def matmul(A, B):
  return A @ B

In [92]:
def hadamard_projection(A, k, p=5):
    m, n = A.shape
    power = 1
    l = k + p
    start = time.time()
    Y = A
    for i in range(0, m):
      Y[i, :] = transforms.fast_hadamard_transform(Y[i, :])
    random_cols = np.random.choice(n, l, replace = False)
    Y = Y[:, random_cols]
    return Y

In [93]:
def randomized_subspace_iteration_with_hadamar(A, k, p=3, q=3):
    _, n = A.shape

    l = k + p
    Y0 = hadamard_projection(A, k, p)

    Q0, _ = np.linalg.qr(Y0)
    Qj = Q0
    for j in range(q):
        Y_tilde_j = A.conj().T @ Qj
        Q_tilde_j, _ = np.linalg.qr(Y_tilde_j)

        Yj = A @ Q_tilde_j
        Qj, _ = np.linalg.qr(Yj)

    return Qj

In [94]:
def randomized_subspace_iteration(A, k, p, q=3, is_custom_mul=False):
    _, n = A.shape
    l = k + p
    Omega = np.random.randn(n, l)
    if is_custom_mul:
        Y0 = my_matmul(A, Omega)
    else:
        Y0 = A @ Omega

    Q0, _ = np.linalg.qr(Y0)
    Qj = Q0
    for j in range(q):
        Y_tilde_j = A.conj().T @ Qj
        Q_tilde_j, _ = np.linalg.qr(Y_tilde_j)

        Yj = A @ Q_tilde_j
        Qj, _ = np.linalg.qr(Yj)

    return Qj

In [95]:
def get_loss(A, Q):
  return np.linalg.norm(Q @ (Q.T @ A) - A) / np.linalg.norm(A)

In [96]:
accuracy = 1e-8

In [97]:
def get_array_projections(A, seed=10):
    m, n = A.shape
    projections = list()
    for i in range(seed):
        rand_vec = np.random.randn(n)
        projection = A @ rand_vec
        projections.append(projection/np.linalg.norm(projection))
    return projections

In [98]:
def get_random_loss(A, projections_arr, Q):
    max_loss = 0
    for projection in projections_arr:
        result = Q @ (Q.T @ projection)
        if np.linalg.norm(result - projection) > max_loss:
            max_loss = np.linalg.norm(result - projection)
    return max_loss
########## !!!!!!!!!!!!!!!!!!!!!!

In [99]:
def check_loss_rand(A, get_Q_func, projections_arr, k, p=3, q=3, max_loss = accuracy):
    Q = get_Q_func(A, k, p, q)
    loss = get_random_loss(A, projections_arr, Q)
    return loss < max_loss

In [100]:
def check_loss_k(A, get_Q_func, k, p=3, q=3, max_loss = accuracy):
    Q = get_Q_func(A, k, p, q)
    loss = get_loss(A, Q)
    return loss < max_loss

In [101]:
def get_rank_binary_search(A, left, right, get_Q_func, p=5, q=3, max_loss = accuracy):
  while right - left > 1: 
    k = (right + left) // 2
    if (check_loss_k(A, get_Q_func, k, p, q, max_loss)):
      right = k
    else:
      left = k
  return right

In [102]:
def get_rank(A, get_Q_func, p=5, q=3, debug=False, max_loss=accuracy):
  k = 1
  while True:
    Q = get_Q_func(A, k, p, q)
    loss = get_loss(A, Q)
    if debug:
      print(f'now k = {k}, now loss = {loss}')
    if (check_loss_k(A, get_Q_func, k, p, q, max_loss)):
      return get_rank_binary_search(A, k // 2, k, get_Q_func, p, q, max_loss)
    k *= 2


In [103]:
def get_rand_rank(A, get_Q_func, p=5, q=3, seed=10, debug=False max_loss=accuracy):
    projections_arr = get_array_projections(A, seed)
    k = 1
    while True:
        Q = get_Q_func(A, k, p, q)
        loss = get_loss(A, Q)
        if debug:
            print(f'now k = {k}, now loss = {loss}')
        if (check_loss_rand(A, get_Q_func, projections_arr, k, p, q, max_loss)):
            return get_rank_binary_search(A, k // 2, k, get_Q_func, p, q, max_loss)
        k *= 2

SyntaxError: invalid syntax. Perhaps you forgot a comma? (422846733.py, line 1)

In [75]:
# def get_rank(A, get_Q, p=5, q=3, max_loss = 1e-14):
#   k = 1
#   while True:
#     Q = get_Q(A, k, p, q)
#     loss = get_loss(A, Q)
#     print(f'now k = {k}, now loss = {loss}')
#     if loss > max_loss:
#       k *= 2
#     else:
#       return k


In [76]:
def random_rank_k_matrix(m, n, k, random_state=None):
    B = np.random.randn(m, k)
    C = np.random.randn(k, n)
    A: np.matrix = B @ C
    return A

In [77]:
k = 45
p = 5
q = 3

In [78]:
m, n = 2**11, 2**11

In [79]:
A = random_rank_k_matrix(m, n, k)

In [80]:
def get_time(A, iteration):
    time_start = time.time()
    rank = get_rank(A, randomized_subspace_iteration_with_hadamar)
    time_finish = time.time()
    return (time_finish - time_start, rank)

In [81]:
def get_rand_time(A, iteration, seed=10):
    time_start = time.time()
    rank = get_rand_rank(A, randomized_subspace_iteration_with_hadamar, seed)
    time_finish = time.time()
    return (time_finish - time_start, rank)

In [ ]:
time_start = time.time()
rank = get_rank(A, randomized_subspace_iteration_with_hadamar)
time_finish = time.time()
print(rank)
print(f'time = {time_finish - time_start}')
randomized_subspace_iteration_with_hadamar_time = time_finish - time_start

now k = 1, now loss = 0.9133747996768127
now k = 2, now loss = 0.9033740645379327
now k = 4, now loss = 0.8712480707263939
now k = 8, now loss = 0.8128935407405258
now k = 16, now loss = 0.6877652372716591
now k = 32, now loss = 0.37995308142795936
now k = 64, now loss = 1.649376872327185e-15
40
time = 1.1790518760681152


In [83]:
# time_start = time.time()
# rank = get_rand_rank(A, randomized_subspace_iteration_with_hadamar)
# time_finish = time.time()
# print(rank)
# print(f'time = {time_finish - time_start}')
# randomized_subspace_iteration_with_hadamar_rand_time = time_finish - time_start

In [ ]:
time_start = time.time()
rank = get_rank(A, randomized_subspace_iteration)
time_finish = time.time()
print(rank)
print(f'time = {time_finish - time_start}')
randomized_subspace_iteration_time = time_finish - time_start

now k = 1, now loss = 0.9151556379579225
now k = 2, now loss = 0.901743277769439
now k = 4, now loss = 0.8719032546845209
now k = 8, now loss = 0.8120947862512847
now k = 16, now loss = 0.6886307007152651
now k = 32, now loss = 0.38032672203550294
now k = 64, now loss = 1.6008232674018017e-15
40
time = 1.0397472381591797


In [ ]:
time_start = time.time()
rank = get_rand_rank(A, randomized_subspace_iteration)
time_finish = time.time()
print(rank)
print(f'time = {time_finish - time_start}')
randomized_subspace_iteration_rand_time = time_finish - time_start

now k = 1, now loss = 0.9137188240596346
now k = 2, now loss = 0.9004796052859182
now k = 4, now loss = 0.8707375460341552
now k = 8, now loss = 0.8145437067902018
now k = 16, now loss = 0.6901700132858581
now k = 32, now loss = 0.37653746693289714
now k = 64, now loss = 1.6314225735181543e-15
40
time = 0.9073400497436523


In [86]:
time_start = time.time()
np_rank = np.linalg.matrix_rank(A)
time_finish = time.time()
print(np_rank)
print(f'time = {time_finish - time_start}')
classic_rank_time = time_finish - time_start

45
time = 1.1004960536956787


In [87]:
print(randomized_subspace_iteration_with_hadamar_time)
# print(randomized_subspace_iteration_with_hadamar_rand_time)
print(randomized_subspace_iteration_time)
print(randomized_subspace_iteration_rand_time)
print(classic_rank_time)

1.1790518760681152
1.0397472381591797
0.9073400497436523
1.1004960536956787


In [88]:
from tqdm import tqdm

all_time_subspace_rand = 0
all_time_classic = 0
for i in tqdm(range(100), desc="running"):
    
    all_time_classic += get_time(A, np.linalg.matrix_rank)[0]
    all_time_subspace_rand += get_rand_time(A, randomized_subspace_iteration)[0]
    
print(f"classic rank = {all_time_classic}")
print (f"our fast rank = {all_time_subspace_rand}")


running:   0%|          | 0/100 [00:00<?, ?it/s]

now k = 1, now loss = 0.9145679859198602
now k = 2, now loss = 0.8999183925706172
now k = 4, now loss = 0.8725238173510893
now k = 8, now loss = 0.8138638944942281
now k = 16, now loss = 0.6893093347084011
now k = 32, now loss = 0.3773357986145922
now k = 64, now loss = 1.8457590687961494e-15
now k = 1, now loss = 0.8429638974898577
now k = 2, now loss = 0.828435956343605
now k = 4, now loss = 0.7964719412587948
now k = 8, now loss = 0.7363484792963775
now k = 16, now loss = 0.604367773771821
now k = 32, now loss = 0.22896100799462463
now k = 64, now loss = 1.984082609456304e-15


running:   1%|          | 1/100 [00:02<03:54,  2.37s/it]

now k = 1, now loss = 0.9158595918295407
now k = 2, now loss = 0.8988031917637334
now k = 4, now loss = 0.8730427834880797
now k = 8, now loss = 0.8134373191396836
now k = 16, now loss = 0.6856746151974693
now k = 32, now loss = 0.38131842371911745
now k = 64, now loss = 1.9894279978397975e-15
now k = 1, now loss = 0.840723999555106
now k = 2, now loss = 0.8278972566826655
now k = 4, now loss = 0.7973503346653497
now k = 8, now loss = 0.7330200053339667
now k = 16, now loss = 0.6027247336791794
now k = 32, now loss = 0.23273018062221162
now k = 64, now loss = 2.279664319394286e-15


running:   2%|▏         | 2/100 [00:04<03:50,  2.35s/it]

...

now k = 1, now loss = 0.9149974669835886
now k = 2, now loss = 0.8998885732402866
now k = 4, now loss = 0.8706524215339563
now k = 8, now loss = 0.8149918964022193
now k = 16, now loss = 0.6864372244152241
now k = 32, now loss = 0.37986173544049795
now k = 64, now loss = 2.5091499143725397e-14
now k = 1, now loss = 0.8409324720236202
now k = 2, now loss = 0.8277184000000288
now k = 4, now loss = 0.8000924012105719
now k = 8, now loss = 0.735608359654738
now k = 16, now loss = 0.6039891869899404
now k = 32, now loss = 0.22936906016659253
now k = 64, now loss = 2.508372627781588e-14


running:  99%|█████████▉| 99/100 [04:09<00:02,  2.94s/it]

now k = 1, now loss = 0.9137355488171791
now k = 2, now loss = 0.9031306460961043
now k = 4, now loss = 0.8713625619683448
now k = 8, now loss = 0.8124236028392384
now k = 16, now loss = 0.6874332114454413
now k = 32, now loss = 0.3803802275933591
now k = 64, now loss = 2.5322126785561878e-14
now k = 1, now loss = 0.8417742662015906
now k = 2, now loss = 0.827786766415479
now k = 4, now loss = 0.7967692398256822
now k = 8, now loss = 0.7359163571492744
now k = 16, now loss = 0.6046324199122535
now k = 32, now loss = 0.22728589086507706
now k = 64, now loss = 2.5325392453741535e-14


running: 100%|██████████| 100/100 [04:11<00:00,  2.52s/it]

123.92225670814514
127.7779450416565
